In [352]:
import pandas as pd
import datetime
import numpy as np
from epiweeks import Week,Year

In [353]:
weather_df = pd.read_csv('../assets/daily_weather_2014_to_2018.csv')
weather_df.head()

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,Admiralty,2014,1,1,0.0,0.0,0.0,0.0,26.3,28.8,24.3,10.8,34.2
1,Admiralty,2014,1,2,0.0,0.0,0.0,0.0,26.9,30.7,24.6,11.5,38.2
2,Admiralty,2014,1,3,4.6,4.4,4.4,4.6,26.5,29.5,24.9,9.7,34.2
3,Admiralty,2014,1,4,3.8,3.8,3.8,3.8,26.4,31.2,24.7,8.1,32.8
4,Admiralty,2014,1,5,3.6,1.2,1.6,2.0,24.6,25.3,23.6,8.4,27.0


In [354]:
dengue_df = pd.read_csv('../assets/WeeklyNumberofDengueandDengueHaemorrhagicFeverCases.csv')
dengue_df.head()

,year,eweek,type_dengue,number
0,2014,1,Dengue,436.0
1,2014,1,DHF,1.0
2,2014,2,Dengue,479.0
3,2014,2,DHF,0.0
4,2014,3,Dengue,401.0


In [355]:
google_trend_df  = pd.read_csv('../assets/gTrends_dengue_2014-2018.csv')
#google_trend_df = google_trend_df.reset_index()
google_trend_df = pd.DataFrame(google_trend_df)[1:].reset_index()
google_trend_df.rename(columns={'index':'Date','Category: All categories':'Search Interest'}, inplace=True)
google_trend_df.head()

,Date,Search Interest
0,2013-12-29,48
1,2014-01-05,60
2,2014-01-12,60
3,2014-01-19,42
4,2014-01-26,41


### Data Cleaning

### Dealing with NA values

In [356]:
weather_df.isnull().sum()

Station                            0
Year                               0
Month                              0
Day                                0
Daily Rainfall Total (mm)          0
Highest 30 Min Rainfall (mm)       0
Highest 60 Min Rainfall (mm)       0
Highest 120 Min Rainfall (mm)      0
Mean Temperature (°C)              0
Maximum Temperature (°C)         129
Minimum Temperature (°C)         141
Mean Wind Speed (km/h)             0
Max Wind Speed (km/h)            167
dtype: int64

In [357]:
weather_df.shape

(107392, 13)

##### Dropping Na value is the misisng dataset is small as compared the whole dataset. 

In [358]:
weather_df.dropna(inplace=True)

In [359]:
weather_df.shape

(107209, 13)

In [360]:
print(f'Rows will NA values only take up {round(((107392-107209)/107392) *100,2)} %')

Rows will NA values only take up 0.17 %


### Dengue_df

In [361]:
dengue_df.shape

(530, 4)

In [362]:
dengue_df.isnull().sum()

year           0
eweek          0
type_dengue    0
number         8
dtype: int64

In [363]:
# Find all 8 rows with na in number column

dengue_df[dengue_df.isnull().any(axis=1)]

,year,eweek,type_dengue,number
210,2015,53,Dengue,NaN
211,2015,53,DHF,NaN
316,2016,53,Dengue,NaN
317,2016,53,DHF,NaN
422,2017,53,Dengue,NaN
423,2017,53,DHF,NaN
528,2018,53,Dengue,NaN
529,2018,53,DHF,NaN


All the Na values belong to week 53, will proceed to drop these rows.

In [364]:
dengue_df.dropna(inplace=True)

In [365]:
dengue_df.shape

(522, 4)

In [366]:
# Do we have other week 53 which have data?
dengue_df[dengue_df['eweek'] == 53]

,year,eweek,type_dengue,number
104,2014,53,Dengue,158.0
105,2014,53,DHF,0.0


In [367]:
# Dropping 2014 eweek 53 too to ensure consistance in our data
dengue_df = dengue_df[dengue_df['eweek'] != 53]

In [368]:
dengue_df.shape

(520, 4)

In [369]:
google_trend_df.isnull().sum()

Date               0
Search Interest    0
dtype: int64

In [370]:
weather_df.dtypes

Station                          object
Year                              int64
Month                             int64
Day                               int64
Daily Rainfall Total (mm)        object
Highest 30 Min Rainfall (mm)     object
Highest 60 Min Rainfall (mm)     object
Highest 120 Min Rainfall (mm)    object
Mean Temperature (°C)            object
Maximum Temperature (°C)         object
Minimum Temperature (°C)         object
Mean Wind Speed (km/h)           object
Max Wind Speed (km/h)            object
dtype: object

In [371]:
# Get all the columns name
weather_df.columns

Index(['Station', 'Year', 'Month', 'Day', 'Daily Rainfall Total (mm)',
       'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
       'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)',
       'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
       'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'],
      dtype='object')

In [372]:
# Need to convert all variabbles to numberic variables

weather_conditon_cols = [ 'Daily Rainfall Total (mm)',
       'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
       'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)',
       'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
       'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)']


for i in weather_conditon_cols:
    weather_df[i] = pd.to_numeric(weather_df[i], errors='coerce')


weather_df.head(5)

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,Admiralty,2014,1,1,0.0,0.0,0.0,0.0,26.3,28.8,24.3,10.8,34.2
1,Admiralty,2014,1,2,0.0,0.0,0.0,0.0,26.9,30.7,24.6,11.5,38.2
2,Admiralty,2014,1,3,4.6,4.4,4.4,4.6,26.5,29.5,24.9,9.7,34.2
3,Admiralty,2014,1,4,3.8,3.8,3.8,3.8,26.4,31.2,24.7,8.1,32.8
4,Admiralty,2014,1,5,3.6,1.2,1.6,2.0,24.6,25.3,23.6,8.4,27.0


In [373]:
# Convert 'Year', 'Month', and 'Day' columns into a datetime object
weather_df['Date'] = pd.to_datetime(weather_df[['Year', 'Month', 'Day']])

# Extract EpiWeek from the 'Date' column
weather_df['Year_eWeek'] = weather_df['Date'].apply(lambda x: Week.fromdate(x))
weather_df['eWeek'] = weather_df['Year_eWeek'].astype(str).str[-2:]
weather_df['eWeek'] = weather_df['eWeek'].astype(int)

In [374]:
weather_df.head(5)

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Date,Year_eWeek,eWeek
0,Admiralty,2014,1,1,0.0,0.0,0.0,0.0,26.3,28.8,24.3,10.8,34.2,2014-01-01,201401,1
1,Admiralty,2014,1,2,0.0,0.0,0.0,0.0,26.9,30.7,24.6,11.5,38.2,2014-01-02,201401,1
2,Admiralty,2014,1,3,4.6,4.4,4.4,4.6,26.5,29.5,24.9,9.7,34.2,2014-01-03,201401,1
3,Admiralty,2014,1,4,3.8,3.8,3.8,3.8,26.4,31.2,24.7,8.1,32.8,2014-01-04,201401,1
4,Admiralty,2014,1,5,3.6,1.2,1.6,2.0,24.6,25.3,23.6,8.4,27.0,2014-01-05,201402,2


In [375]:
weather_df[weather_df['Date'] == '2014-12-29'].head(1)

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Date,Year_eWeek,eWeek
332,Admiralty,2014,12,29,1.0,1.0,1.0,1.0,25.2,28.2,24.1,13.3,50.0,2014-12-29,201453,53


In [376]:

### Aggregate weather information by year and weeks to match dengue cases format
weather_df_cleaned = weather_df.groupby(['Year','eWeek','Station']).agg({
    'Daily Rainfall Total (mm)' : np.sum,
    'Highest 30 Min Rainfall (mm)':np.max,
    'Highest 60 Min Rainfall (mm)':np.max,
    'Highest 120 Min Rainfall (mm)':np.max,
    'Mean Temperature (°C)':np.mean,
    'Maximum Temperature (°C)':np.max,
    'Minimum Temperature (°C)':np.max,
    'Mean Wind Speed (km/h)':np.mean,
    'Max Wind Speed (km/h)':np.max}).reset_index()


weather_df_cleaned = weather_df_cleaned.groupby(['Year','eWeek']).agg({
    'Daily Rainfall Total (mm)' : np.mean,
    'Highest 30 Min Rainfall (mm)':np.mean,
    'Highest 60 Min Rainfall (mm)':np.mean,
    'Highest 120 Min Rainfall (mm)':np.mean,
    'Mean Temperature (°C)':np.mean,
    'Maximum Temperature (°C)':np.mean,
    'Minimum Temperature (°C)':np.mean,
    'Mean Wind Speed (km/h)':np.mean,
    'Max Wind Speed (km/h)':np.mean}).reset_index()

#weather_df_cleaned['Year_Week'] =  weather_df_cleaned['Year'].astype(str) + '_' + weather_df_cleaned['Week'].astype(str)

In [377]:
weather_df_cleaned.head(3)

,Year,eWeek,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2014,1,12.967797,8.641818,10.209091,11.001818,26.932018,31.615,25.110,8.771250,38.394737
1,2014,2,57.510169,12.750909,15.549091,18.998182,26.333960,32.165,24.705,7.247024,40.494737
2,2014,3,2.672881,2.320000,2.418182,2.432727,26.321053,30.255,25.000,12.207857,43.347368


In [378]:
#Save to csv file
weather_df_cleaned.to_csv('../assets/sg_weather_df_by_weeks.csv', index=False)

In [379]:
# Combine Dengue and DHF together by Year and week
dengue_df = dengue_df.groupby(['year','eweek']).agg({'number':np.sum}).reset_index()

# Rename year = Year, eweek = Week and number = Cases
# Assuming df is your DataFrame
dengue_df.rename(columns={'year': 'Year', 'eweek': 'eWeek', 'number': 'Cases'}, inplace=True)

In [380]:
# Merge dengue_df with cleaned_weather_df
dengue_w_weather_df =dengue_df.merge(weather_df_cleaned, how = 'left', on = ['Year','eWeek'])
dengue_w_weather_df['Year_eWeek'] = dengue_w_weather_df['Year'].astype(str) + '-' + dengue_w_weather_df['eWeek'].astype(str) 

def convert_to_date(year_eweek):
    year, eweek = map(int, year_eweek.split('-'))
    return Week(year, eweek).startdate()

# Apply the function to the DataFrame
dengue_w_weather_df['Date'] = dengue_w_weather_df['Year_eWeek'].apply(convert_to_date)

dengue_w_weather_df['Date'] = pd.to_datetime(dengue_w_weather_df['Date'])

#Extract the month from the 'Date' column
dengue_w_weather_df['Month'] = dengue_w_weather_df['Date'].dt.month

# Create 'Year_Month' column by combining 'Year' and 'Month'
dengue_w_weather_df['Year_Month'] = dengue_w_weather_df['Year'].astype(str) + '-' + dengue_w_weather_df['Month'].astype(str)

dengue_w_weather_df.head()

,Year,eWeek,Cases,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year_eWeek,Date,Month,Year_Month
0,2014,1,437.0,12.967797,8.641818,10.209091,11.001818,26.932018,31.615,25.110,8.771250,38.394737,2014-1,2013-12-29,12,2014-12
1,2014,2,479.0,57.510169,12.750909,15.549091,18.998182,26.333960,32.165,24.705,7.247024,40.494737,2014-2,2014-01-05,1,2014-1
2,2014,3,401.0,2.672881,2.320000,2.418182,2.432727,26.321053,30.255,25.000,12.207857,43.347368,2014-3,2014-01-12,1,2014-1
3,2014,4,336.0,0.000000,0.000000,0.000000,0.000000,25.869173,30.035,23.880,12.107857,41.184211,2014-4,2014-01-19,1,2014-1
4,2014,5,234.0,0.000000,0.000000,0.000000,0.000000,26.200902,32.530,24.240,12.105714,42.563158,2014-5,2014-01-26,1,2014-1


### Google Trend 

In [381]:
google_trend_df.dtypes

Date               object
Search Interest    object
dtype: object

In [382]:
# Extract EpiWeek from the 'Date' column

google_trend_df['Date'] = pd.to_datetime(google_trend_df['Date'])


google_trend_df.head()

,Date,Search Interest
0,2013-12-29,48
1,2014-01-05,60
2,2014-01-12,60
3,2014-01-19,42
4,2014-01-26,41


In [383]:
dengue_w_weather_df =dengue_w_weather_df.merge(google_trend_df, how = 'left', on = ['Date'])

In [384]:
dengue_w_weather_df.shape

(260, 17)

In [385]:
dengue_w_weather_df.head()

,Year,eWeek,Cases,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year_eWeek,Date,Month,Year_Month,Search Interest
0,2014,1,437.0,12.967797,8.641818,10.209091,11.001818,26.932018,31.615,25.110,8.771250,38.394737,2014-1,2013-12-29,12,2014-12,48
1,2014,2,479.0,57.510169,12.750909,15.549091,18.998182,26.333960,32.165,24.705,7.247024,40.494737,2014-2,2014-01-05,1,2014-1,60
2,2014,3,401.0,2.672881,2.320000,2.418182,2.432727,26.321053,30.255,25.000,12.207857,43.347368,2014-3,2014-01-12,1,2014-1,60
3,2014,4,336.0,0.000000,0.000000,0.000000,0.000000,25.869173,30.035,23.880,12.107857,41.184211,2014-4,2014-01-19,1,2014-1,42
4,2014,5,234.0,0.000000,0.000000,0.000000,0.000000,26.200902,32.530,24.240,12.105714,42.563158,2014-5,2014-01-26,1,2014-1,41


In [386]:
dengue_w_weather_df.isnull().sum()

Year                             0
eWeek                            0
Cases                            0
Daily Rainfall Total (mm)        0
Highest 30 Min Rainfall (mm)     0
Highest 60 Min Rainfall (mm)     0
Highest 120 Min Rainfall (mm)    0
Mean Temperature (°C)            0
Maximum Temperature (°C)         0
Minimum Temperature (°C)         0
Mean Wind Speed (km/h)           0
Max Wind Speed (km/h)            0
Year_eWeek                       0
Date                             0
Month                            0
Year_Month                       0
Search Interest                  0
dtype: int64

In [387]:
#Save to csv file
dengue_w_weather_df.to_csv('../assets/sg_dengue_w_weather_df.csv', index=False)